In [ ]:
# Installing required libs
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [ ]:
!wget -O master.zip https://github.com/ryanzhumich/AESLC/archive/refs/heads/master.zip
%rm -rf /content/master/

import zipfile
import os

def extract_zip(zip_path, extract_to):
    # Ensure the extraction directory exists
    os.makedirs(extract_to, exist_ok=True)

    # Open the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents to the specified directory
        zip_ref.extractall(extract_to)

zip_path = '/content/master.zip'
extract_to = '/content/master'
extract_zip(zip_path, extract_to)

--2024-07-21 05:06:04--  https://github.com/ryanzhumich/AESLC/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ryanzhumich/AESLC/zip/refs/heads/master [following]
--2024-07-21 05:06:04--  https://codeload.github.com/ryanzhumich/AESLC/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.116.9
Connecting to codeload.github.com (codeload.github.com)|140.82.116.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [    <=>             ]  11.11M  14.2MB/s    in 0.8s    

2024-07-21 05:06:05 (14.2 MB/s) - ‘master.zip’ saved [11650497]



In [ ]:
import glob

train_directory_path  = '/content/master/AESLC-master/enron_subject_line/train'
dev_directory_path  = '/content/master/AESLC-master/enron_subject_line/dev'
test_directory_path  = '/content/master/AESLC-master/enron_subject_line/test'

def load_files(directory, has_annotations=False):
    files = glob.glob(os.path.join(directory, "*"))
    data = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as file:
            content = file.read()
        email_body = ""
        subject_line = ""
        email_body, subjects = content.split('@subject\n', 1)
        annotations = []
        if has_annotations:
          subjects = subjects.replace("@ann0", "@subject\n").replace("@ann1", "@subject\n").replace("@ann2", "@subject\n")
          subject_line, annotation_1, annotation_2, annotation_3 = subjects.split('@subject\n')
          subject_line = subject_line.strip()
          annotations = [annotation_1.strip(), annotation_2.strip(), annotation_3.strip()]
          data.append((email_body, subject_line, annotations))
        else:
          subject_line = subjects.strip()
          data.append((email_body, subject_line))
    return data

# Load the data
dev_data = load_files(dev_directory_path, has_annotations=True)
train_data = load_files(train_directory_path)
test_data = load_files(test_directory_path, has_annotations=True)

# Debugging: Check dataset lengths
print(f"Train dataset length: {len(train_data)}")
print(f"Dev dataset length: {len(dev_data)}")
print(f"Test dataset length: {len(test_data)}")


Train dataset length: 14436
Dev dataset length: 1960
Test dataset length: 1906


In [ ]:
train_data[:2]

[('Frank Ermis -  Jay Reitmeyer - Eastern Rockies lead.\nMatt Smith - Trading Track, west fundies, Socal cash trading, scheduling, and risk.\nJason Wolfe - Trading Track, west fundies, NoCal cash trading, scheduling, and risk.\nSusan Scott - Rockies scheduling, cash trading, and risk experience.\nKeith Holst - West Fundies lead.\nTori Kuykendall - Southwest knowledge, cash trading, scheduler, fundies, basis.\nThis list represents critical people to continue making markets, fundies analysis, and trading.\nGrigsby\n\n',
  'Critical'),
 ("Move-in date for us is December 15th and your assistance is desired for selection of fabrics for KLL's furnishings.\nYou will appreciate that time is of the essence to be certain  his office is ready for move-in.\nPlease let me know, or you can contact Gary Napper directly at 713/345-3984 as to your earliest availability for meeting with him and the designer and architect, Jeffrey State.\nThey will tailor their schedule around yours.\nThanks Linda.\n\n",

In [ ]:
train_data[0][1], dev_data[0]

('Gas Indices',
 ('Tana,  Per my voice mail, I received a call from Gregg Thompson with Coast energy Canada, and US.\nHe has several questions regarding the base contract that they have with Enron, and EnronOnline.\nFor example, if their users logged into EOL under Coast Canada, and transacted on an US product, what underlying agreement would be governed?\nFrom my conversation with Gregg, I believe that they are trying to figure out  what contract applies to what entity, when transacted on EOL.\nIf you could give them a call they would really appreciate it.\nThanks for your time.\n\n',
  'Coast Energy Canada, and Coast Energy Group contract question?',
  ['call from gregg thompson with coast energy canada',
   'coast energy canada',
   'answers needed for gregg thompson with coast energy']))

In [ ]:
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
import torch
import pprint
import evaluate
import numpy as anp

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

MODEL = 't5-small'
BATCH_SIZE = 12
NUM_PROCS = 4
EPOCHS = 2
OUT_DIR = 'results_t5base'
MAX_LENGTH = 512 # Maximum context length to consider while preparing dataset.

tokenizer = T5Tokenizer.from_pretrained(MODEL)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from torch.utils.data import Dataset
class EmailDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        email_body, subject_line = self.data[index][:2]
        prompt = f"summarize: {email_body}"

        # Tokenize the prompt and the target subject line
        inputs = self.tokenizer(prompt, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        labels = self.tokenizer(subject_line, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")

        # Squeeze the tensors to remove the batch dimension
        input_ids = inputs["input_ids"].squeeze(0)
        attention_mask = inputs["attention_mask"].squeeze(0)
        labels = labels["input_ids"].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

train_data_tokenized = EmailDataset(train_data, tokenizer )
dev_data_tokenized = EmailDataset(dev_data, tokenizer)
test_data_tokenized = EmailDataset(test_data, tokenizer)

In [ ]:
for email in train_data_tokenized:
  print(email)
  break

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

In [ ]:
inputs = tokenizer.encode(
    "summarize: " + train_data[0][0],
    return_tensors='pt',
    max_length=512,
    truncation=True
)
inputs = inputs.to(device)
inputs

tensor([[21603,    10,  4937,   848,    51,   159,     3,    18,  9373,   419,
           155,   526,  7975,     3,    18,  7345,  3120,   725,   991,     5,
          5199,  3931,     3,    18, 15088,  8799,     6,  4653,  3069,   725,
             6,   264,  1489,  1723,  3415,     6, 16131,     6,    11,  1020,
             5,  9637,  9091,    15,     3,    18, 15088,  8799,     6,  4653,
          3069,   725,     6,   465, 14318,  1723,  3415,     6, 16131,     6,
            11,  1020,     5, 10445,  4972,     3,    18,  3120,   725, 16131,
             6,  1723,  3415,     6,    11,  1020,   351,     5, 17017,  5838,
             7,    17,     3,    18,  1244,  3563,   725,   991,     5,  3794,
            23,  3695,    63,  2217,    26,  1748,     3,    18, 21423,  1103,
             6,  1723,  3415,     6,  2023,    52,     6,  3069,   725,     6,
          1873,     5,   100,   570,  5475,  2404,   151,    12,   916,   492,
          3212,     6,  3069,   725,  1693,     6,  

In [ ]:
summary_ids = model.generate(
  train_data_tokenized[0]['input_ids'].unsqueeze(0).to(device),
  #inputs,
  max_length=50,
  num_beams=5,
  early_stopping=True,
)
print(f"email_body : {train_data[0][0]}")
print(f"Actual Subject : {train_data[0][1]}")
print(f"Generated Subject : {tokenizer.decode(summary_ids[0], skip_special_tokens=True)}")

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    #warmup_steps=100,
    #weight_decay=0.01,
    logging_dir=OUT_DIR,
    #logging_steps=10,
    evaluation_strategy='epoch',#'steps',
    #eval_steps=2000,
    save_strategy='epoch',
    save_total_limit=1,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_tokenized,
    eval_dataset=dev_data_tokenized,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

history = trainer.train()

In [ ]:
tokenizer.save_pretrained(OUT_DIR)
!zip -r {OUT_DIR} {OUT_DIR}

In [ ]:
zip_path = '/content/results_t5base.zip'
extract_to = '/content/results_t5base'
extract_zip(zip_path, extract_to)

In [ ]:
model_path = f"/content/{OUT_DIR}/checkpoint-2406"  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(OUT_DIR)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
for i in range(10):
    inputs = tokenizer.encode(
        "summarize: " + test_data[i][0],
        return_tensors='pt',
        max_length=512,
        truncation=True
    ).to(device)

    summary_ids = model.to(device).generate(
      inputs,
      max_length=10,
      num_beams=5,
      early_stopping=True,
    )
    print(f"email_body : {test_data[i][0]}")
    print(f"Actual Subject : {test_data[i][1]}")
    print(f"Generated Subject After training : {tokenizer.decode(summary_ids[0], skip_special_tokens=True)}")
    print('--------------------------------------------------------------------------------------------------------')

**Load a model from GDrive**

In [ ]:
import zipfile
import os
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Pruning**

In [ ]:
import torch.nn.utils.prune as prune
import torch.nn as nn

# Define a helper function to prune a layer
def apply_pruning_to_layer(layer, amount=0.2):
    prune.l1_unstructured(layer, name='weight', amount=amount)
    prune.remove(layer, 'weight')

# Prune all the linear layers in the model
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        apply_pruning_to_layer(module)

# Verify the size reduction
print(f"Model size after pruning: {sum(p.numel() for p in model.parameters() if p.requires_grad)} parameters")

**Quantization**

In [ ]:
import torch.quantization

# Set the model to evaluation mode
model.eval()

# Apply dynamic quantization to the model
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Verify the size reduction
print(f"Model size after quantization: {sum(p.numel() for p in quantized_model.parameters() if p.requires_grad)} parameters")

Model size after quantization: 16466432 parameters


In [ ]:
torch.save(quantized_model, "/content/quantizedt5.pt")

In [ ]:
torch.save(quantized_model, "/content/drive/MyDrive/Colab Notebooks/IIITH-FinalProject/quantized-t5small.pt")  #save to drive

# Load quantized model and test

In [ ]:
#load from drive
model_from_gdrive = torch.load("/content/drive/MyDrive/Colab Notebooks/IIITH-FinalProject/quantized-t5small.pt")
model_from_gdrive.eval()

In [ ]:
#load from colab local
qmodel = torch.load("/content/quantizedt5.pt")
qmodel.eval()

In [ ]:
# this is just for reference
#torch.save(model, PATH) # saves the entire model
#model = torch.load(PATH) # loads the entire model with weights

#torch.save(model.state_dict(), PATH) # saves state_dict
# now loading the state_dict first is needed, after which it can
# be copied into model
#model.load_state_dict(torch.load(PATH))

In [ ]:
#save the model as a state dictionary
#state = {'net_dict': quantized_model.state_dict()}
#torch.save(state, './quantized_t5.t7')

#Load the model
#ckpt = torch.load('./quantized_t5.t7')
#qmodel = quantized_model.load_state_dict(ckpt['net_dict'])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for i in range(10):
    inputs = tokenizer.encode(
        "summarize: " + test_data[i][0],
        return_tensors='pt',
        max_length=512,
        truncation=True
    ).to(device)

    summary_ids = qmodel.to(device).generate(
      inputs,
      max_length=10,
      num_beams=5,
      early_stopping=True,
    )
    print(f"email_body : {test_data[i][0]}")
    print(f"Actual Subject : {test_data[i][1]}")
    print(f"Generated Subject After training : {tokenizer.decode(summary_ids[0], skip_special_tokens=True)}")
    print('--------------------------------------------------------------------------------------------------------')